# Fweet
---

## Setup

In [ ]:
# Import all of our dependencies.
# Note `weasyprint` needs `apt-get install libcairo2 libpango-1.0-0 libpangocairo-1.0-0 libgdk-pixbuf2.0-0 libffi-dev shared-mime-info poppler-utils`
import os
import datetime
import logging
import time
import json
import requests
import pathlib
import numpy
import pandas as pd
from pandas import json_normalize
from typing import List
import phonenumbers
from functools import partial
import sys 


logger = logging.getLogger()
print("======MODE=======")

print(os.environ.get('FWEET_ENV', 'DEV'))
mode = os.environ.get('FWEET_ENV', 'DEV')


# os.environ['FWEET_ENV'] = 'PROD'
# TEMP WORKAROUND
global workspace
if os.environ.get('FWEET_ENV', 'FWEET_ENV') == 'PROD':
     workspace = '%s/processing/notebook' % os.getcwd()
    #  workspace = os.getcwd()

     logger.setLevel(os.environ.get('LOG_LEVEL', logging.INFO))
else:
    workspace = os.getcwd()
    logger.setLevel(os.environ.get('LOG_LEVEL', logging.DEBUG))


workspace

In [ ]:
#make sure you are in DEV mode, otherwise test fails
assert mode == "DEV"
print(mode)

In [ ]:
#default_exp
#export

from TwitterAPI import TwitterAPI
from requests_oauthlib import OAuth1

class Twitter:
    mode = os.environ.get('FWEET_ENV', 'DEV')

    twitter_api_key = os.environ.get('API_KEY', 'WvnBpCtFTtWY0l0mxY15AE7uI')
    twitter_api_secret = os.environ.get('API_SECRET', 'rqQThnTZsYiM35w5ImeOImQ97omTy8ZG27Jt43U1tqBddFLgzk')
    fweet_token = os.environ.get('FWEET_TOKEN', '1383517092553510913-y3pC1SebrZTc50TNRqdDSpZ2jPKSE3')
    fweet_secret = os.environ.get('FWEET_SECRET', 'XjNBnNo7fL7fQ7UPPGUEuNovvAIRxAz9Nn85PRhYSlgLJ')

    api: TwitterAPI # api v2
    auth: OAuth1 # @fweetmachine Post Auth to be used when replying and posting to Twitter API v1
    bearer_token: str

    def __init__(self):
        self.api = TwitterAPI(self.twitter_api_key, self.twitter_api_secret, self.fweet_token, self.fweet_secret)
        self.auth = OAuth1(self.twitter_api_key, self.twitter_api_secret, self.fweet_token, self.fweet_secret)
    
    #not tested
    def load_bearer_token(self, token_resp=None):
        # get the bearer token for the fweet account
        # https://developer.twitter.com/en/docs/authentication/oauth-2-0/bearer-tokens
        payload = {
            'grant_type': 'client_credentials',
        }
        
       
        if (token_resp==None):
            token_resp = requests.post('https://api.twitter.com/oauth2/token',
                auth=(self.twitter_api_key, self.twitter_api_secret), 
                data=payload)
        
        
        if token_resp.status_code != 200:
            print("ERROR")
            logger.error('Error message: %s' % token_resp.reason)
            exit(1)

        token = token_resp.json()
        self.bearer_token = token.get('access_token') # write out as materialization
    
    #Tested
    # https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/api-reference/get-favorites-list
    def fetch_favorites(self, favorites_resp=None):
        headers = {
            'Authorization': 'Bearer %s ' % self.bearer_token
        }
        params = {
            'count': '20'
        }
        if (favorites_resp == None): 
            favorites_resp = requests.get('https://api.twitter.com/1.1/favorites/list.json', params=params, auth=self.auth)
        
    
        if favorites_resp.status_code != 200:
            logger.error('Error message')
            sys.exit(1)

        favorites_json = favorites_resp.json()
        favorites_df = pd.json_normalize(favorites_json)

        return favorites_df
    
    #tested
    def fetch_mentions(self, mentions_resp=None):
        # # Mentions API
        # # https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-mentions
        expansions = 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'
        user_fields = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
        media_fields = 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics,promoted_metrics'
        tweet_fields = 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'
        poll_fields = 'duration_minutes,end_datetime,id,options,voting_status'
        max_results = '100'
        
        #payload contains all of the data that is displayed 
        payload = {
            'expansions': expansions,
            'user.fields': user_fields,
            'media.fields': media_fields,
            'tweet.fields': tweet_fields,
            'poll.fields': poll_fields,
            'max_results': max_results,
            # 'tweet_mode': 'extended',
        }

        headers = {
            'Authorization': 'Bearer %s ' % self.bearer_token
        }

        # temp
        fweet_user_id = '1383517092553510913'

    
        #TODO: add filter for tweets after last execution
        #adding this to check response, setting deafult to None, so it enters this and updates
        if(mentions_resp == None):
            mentions_resp = requests.get('https://api.twitter.com/2/users/%s/mentions' % fweet_user_id, params=payload, headers=headers)
        
        if mentions_resp.status_code != 200:
            logger.error('Error message') #% mentions_resp.reason)

            sys.exit(1)
        # grap the response
        # print(mentions_resp.json())
        
        
        mentions_json = mentions_resp.json()
        mentions = mentions_json['data']



        #take the media tag from the json blob and store in media map
        media = mentions_json['includes']['media']
        media_map = {}
        for m in media:
            media_map[m['media_key']] = m

        #same with users
        users = mentions_json['includes']['users']
        user_map = {}
        for u in users:
            user_map[u['id']] = u
            user_map[u['username']] = u

        # tweets = mentions_json['includes']['tweets']
        # tweet_map = {}
        # for t in tweets:
        #     tweet_map[t['id']] = t

        mentions_df = pd.json_normalize(mentions)
        print(mentions_df)

        #return mentions_df, media and users
        return mentions_df, media, users
    

    

    @staticmethod
    #tested
    #called in run recieved step, where onces the tweet is approved, the row becomes the df with the approved status and token is the twitter bearer token 
    def get_tweet_content(row, bearer_token, tweet_resp=None):
        row['content'] = None

        #do nothing if status not recieved or pending 
        if row['status'] != 'received' and row['status'] != 'pending':
            return row

        headers = {
            'Authorization': 'Bearer %s ' % bearer_token
        }

        expansions = 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'
        user_fields = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
        media_fields = 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics,promoted_metrics'
        tweet_fields = 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'
        poll_fields = 'duration_minutes,end_datetime,id,options,voting_status'

        #fill in the params
        params = {
            'expansions': expansions,
            'user.fields': user_fields,
            'media.fields': media_fields,
            'tweet.fields': tweet_fields,
            'poll.fields': poll_fields,
        }

        # temp
        conversation_id = row['conversation_id']

        #TODO: add filter for tweets after last execution
        if(tweet_resp==None):
            tweet_resp = requests.get('https://api.twitter.com/2/tweets/%s' % conversation_id, params=params, headers=headers)

        if tweet_resp.status_code != 200:
            logger.error('Error message.')
            return row
        # grap the response
        # print(mentions_resp.json())
        tweet_json = tweet_resp.json()
        
        print('==TWEET CONTENT===')
        print(tweet_json)
        
        tweet = tweet_json['data']
        
        row['content'] = tweet['text']
        
        #populate the df
        for user in tweet_json['includes']['users']:
            if user['id'] == tweet['author_id']:
                row['author_name'] = user['name']
                row['author_profile_pic'] = user['profile_image_url']
                row['author_verified'] = user['verified']
                row['author_location'] = user.get('location', 'The Internet.')
                row['author_username'] = user['username']

        row['author_public_metrics.retweet_count'] = str(int(tweet['public_metrics']['retweet_count']))
        row['author_public_metrics.reply_count'] = str(int(tweet['public_metrics']['reply_count']))
        row['author_public_metrics.like_count'] = str(int(tweet['public_metrics']['like_count']))


        row['media'] = []
        if 'attachments' in tweet:
            for media_key in tweet['attachments']['media_keys']:
                for media_obj in tweet_json['includes']['media']:
                    row['media'].append(media_obj)
        
        return row


    @staticmethod
    #tested
    #adds media to the tweet
    def map_media_to_tweet(row, media):
        row['media'] = []
        if 'attachments' in row:
            for media_key in row['attachments']['media_keys']:
                row['media'].append(media_key)
        return row
    
    @staticmethod
    #called in analyze_mentions step, on the tweets where the name is null, means new tweets, it populates the df
    #called with users from mentions
    #tested
    def map_usernames_to_fweet(row, users):
        author_id = row['author_id']
        #create a new empty dict
        row['mention_users'] = {}
        for user in users:
            if user['id'] == author_id:
                row['name'] = user['name']
                row['profile_pic'] = user['profile_image_url']
                row['verified'] = user['verified']
                row['location'] = user.get('location', 'The Internet.')
                row['username'] = user['username']
                row['mention_obj'] = user
            else:
                row['mention_users'][user['username']] = user
        return row


    @staticmethod
    #not tested for PROD
    #upload the media, only in PROD mode
    def upload_media(row, auth, mode):
        if mode == 'PROD':
            files = {"media": open(row['jpg'], 'rb')}
            media_resp = requests.post('https://upload.twitter.com/1.1/media/upload.json', auth=auth, files=files)
            media_json = media_resp.json()
            row['media_id'] = media_json['media_id']
        else:
            row['media_id'] = None

        return row
    
    # https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/api-reference/post-statuses-update
    @staticmethod
    #not tested for PROD
    #actually send the tweet
    def send_tweet(row, auth, mode):
        in_reply_to_status_id = row['id']
        status = '%s' % (row['reply'])
        auto_populate_reply_metadata = True

        #what is returned on Twitter
        payload = {
            'in_reply_to_status_id': in_reply_to_status_id,
            'status': status,
            'auto_populate_reply_metadata': auto_populate_reply_metadata,
        }

        if ('media_id' in row and row['media_id']):
            payload['media_ids'] = row['media_id']

        print("===TWEET===")
        if (mode != 'PROD'):
            print("skipping")
            print(payload)
            return row
        else:
            status_resp = requests.post('https://api.twitter.com/1.1/statuses/update.json', auth=auth, data=payload)

            print(status_resp.json())
            return row
    


# Fetch Mentions()

In [ ]:
from unittest.mock import MagicMock, Mock, PropertyMock
#takes in a twitter object

def test_fetch_mentions():
    #Make a mock of the Twitter Resp object, not testing the Twitter Object or OAuth Object
    #because those are invoked methods
    
    twitter_mock_valid_response = Mock()
    type(twitter_mock_valid_response).status_code = PropertyMock(return_value=200)

    json_file = open("fixtures/mentions.json")
    twitter_mock_valid_response.json = MagicMock(return_value=json.load(json_file))
    json_file.close()
    
    valid_resp_df, valid_media, valid_users = twitter.fetch_mentions(twitter_mock_valid_response) 
    
#     assert valid_resp_df ==             author_id    text  \
# 0  1034147061711679488  @LeBraat @TwitterDev @LeGuud There's enough @t...   
# 1            199566737  Apparently I'm not the only one (of my test ac...   
# 2   930524282358325248           Can I join this @twitterdev love party?!   
# 3  1034147061711679488                    I love me some @twitterdev too!   
# 4            199566737  This is a test, but also a good excuse to expr...   

# lang      conversation_id                   id  
# 0   en  1375152449594523649  1375152598945312768  
# 1   en  1375152449594523649  1375152449594523649  
# 2   en  1375152043455873027  1375152043455873027  
# 3   en  1375151947360174082  1375151947360174082  
# 4   en  1375151827189137412  1375151827189137412  
    
    assert valid_media == []
    assert valid_users == [ {
        
        "name": "LeStache",
        "id": "1034147061711679488",
        "entities": {
          "url": {
            "urls": [
              {
                "start": 0,
                "end": 23,
                "url": "https://t.co/7IDoW8iFLm",
                "expanded_url": "https://twitter.com",
                "display_url": "twitter.com"
              }
            ]
          },
          "description": {
            "urls": [
              {
                "start": 21,
                "end": 44,
                "url": "https://t.co/v6nxjDjZR3",
                "expanded_url": "https://google.com",
                "display_url": "google.com"
              }
            ],
            "hashtags": [
              {
                "start": 15,
                "end": 20,
                "tag": "test"
              }
            ],
            "mentions": [
              {
                "start": 0,
                "end": 8,
                "username": "lebraat"
              }
            ],
            "cashtags": [
              {
                "start": 9,
                "end": 14,
                "tag": "twtr"
              }
            ]
          }
        },
        "username": "LeStaache",
        "created_at": "2018-08-27T18:34:07.000Z"
      } ]

   
    twitter_mock_invalid_response = Mock()
    type(twitter_mock_invalid_response).status_code = PropertyMock(return_value=400)
    
    #TESTING INVALID RESPONSE - UNCOMMENT AT THE END

    # invalid_response = twitter.fetch_mentions(twitter_mock_invalid_response)  #checking invalid resp
    # # #assert that the function will fail
    # invalid_response.assertRaises(SystemExit())

    
def test_fetch_favorites():

    twitter_mock_valid_response = Mock()
    type(twitter_mock_valid_response).status_code = PropertyMock(return_value=200)

    json_file = open("fixtures/favorites.json")
    twitter_mock_valid_response.json = MagicMock(return_value=json.load(json_file))
    json_file.close()

    valid_response = twitter.fetch_favorites(twitter_mock_valid_response)
#     assert valid_response ==                        created_at                   id               id_str  \
# 0  Tue May 14 17:58:31 +0000 2019  1128358947772145672  1128358947772145672   

#                                                 text  truncated  \
# 0  Through the Twitter Developer Labs program, we...       True   

#   in_reply_to_status_id in_reply_to_status_id_str in_reply_to_user_id  \
# 0                  None                      None                None   

#   in_reply_to_user_id_str in_reply_to_screen_name  ...  \
# 0                    None                    None  ...   

#   quoted_status.coordinates quoted_status.place quoted_status.contributors  \
# 0                      None                None                       None   

#   quoted_status.is_quote_status  quoted_status.retweet_count  \
# 0                         False                          169   

#    quoted_status.favorite_count quoted_status.favorited  \
# 0                           350                   False   

#    quoted_status.retweeted  quoted_status.possibly_sensitive  \
# 0                    False                             False   

#    quoted_status.lang  
# 0                  en 

    twitter_mock_invalid_response = Mock()
    type(twitter_mock_invalid_response).status_code = PropertyMock(return_value=400)    

    #TESTING INVALID RESPONSE - UNCOMMENT AT THE END
    #invalid_response = twitter.fetch_mentions(twitter_mock_invalid_response)  #checking invalid resp
    ##assert that the function will fail
    #invalid_response.assertRaises(SystemExit())


def test_get_tweet_content(): 
  #again not going to test the twitter method because invoked
  row_test_fixture_pending = {"status" : "pending", "conversation_id" : 1234}
  row_test_fixture_received = {"status" : "received", "conversation_id" : 1234}

  row_fixture_not = {"status" : "not_received"} 
  bearer_token = twitter.bearer_token

  twitter_mock_valid_response = Mock()
  type(twitter_mock_valid_response).status_code = PropertyMock(return_value=200)

  #should just return the same thing
  row_test = twitter.get_tweet_content(row_fixture_not, bearer_token,twitter_mock_valid_response)
  assert row_test == {'status': 'not_received', 'content': None}

  twitter_mock_invalid_response = Mock()
  type(twitter_mock_invalid_response).status_code = PropertyMock(return_value=400)

  #should return the row with status == pending
  fail_resp_test = twitter.get_tweet_content(row_test_fixture_pending, bearer_token, twitter_mock_invalid_response)
  assert fail_resp_test == {'status': 'pending', 'conversation_id': 1234, 'content': None}

  #making the user_id == author_id so we can test the if statement
  json_file = open("fixtures/if_true_tweets.json")
  twitter_mock_valid_response.json = MagicMock(return_value=json.load(json_file))
  json_file.close()

  #making the user_id == author_id so we can test the if statement

  pass_resp_test = twitter.get_tweet_content(row_test_fixture_pending, bearer_token, twitter_mock_valid_response)
  
  #user['id'] == tweet['author_id]: 
  assert pass_resp_test.get("author_name") == "test_name"
  assert pass_resp_test.get("author_profile_pic") == "https://test_url.com"
  assert pass_resp_test.get("author_verified") == "yes"
  assert pass_resp_test.get("author_location") == "test_location"
  assert pass_resp_test.get("author_username") == "test_username"
  #this int also makes sure we test that the type casting of the algo
  assert pass_resp_test.get("author_public_metrics.retweet_count") == "107"
  assert pass_resp_test.get("author_public_metrics.reply_count") == "31"
  assert pass_resp_test.get("author_public_metrics.like_count") == "258"
  #append the media object
  assert pass_resp_test.get("media") == [{'media_key': '7_1138489597158199298', 'type': 'video'}]


  twitter_mock_valid_else_response = Mock()
  type(twitter_mock_valid_else_response).status_code = PropertyMock(return_value=200)

  json_file = open("fixtures/not_true_tweets.json")
  twitter_mock_valid_else_response.json = MagicMock(return_value=json.load(json_file))
  json_file.close()

  #this condition is false user['id'] == tweet['auther_id]: 
  pass_resp_different_id = twitter.get_tweet_content(row_test_fixture_pending, bearer_token, twitter_mock_valid_else_response)
  
  #won't have the user author data, only media data, and row metrics data

  # assert pass_resp_different_id == {'status': 'pending', 'conversation_id': 1234, 'content': '🎺 da-dada-DAH! We’re introducing the first Twitter Developer Labs endpoints: \n\n✨GET/users and GET/tweets ✨\n\nLabs is now open to all developers to start experimenting today 👉 https://t.co/eNx4Wc3Qwj https://t.co/ucmZrJAYjk', 'author_public_metrics.retweet_count': '107', 'author_public_metrics.reply_count': '31', 'author_public_metrics.like_count': '258', 'media': [{'media_key': '7_1138489597158199298', 'type': 'video'}]}



#this method is never used
def test_map_media_to_tweet(): 
    if_row_fixture = {"attachments" : {"media_keys": "123456"}}
    else_row_fixture = {}

    #should just return [] attached to media
    media = ""
    else_condition = twitter.map_media_to_tweet(else_row_fixture, media)
    assert else_condition == {'media': []}

    #should return media key added to the media row
    if_condition = twitter.map_media_to_tweet(if_row_fixture, media)
    #is this correct?

    assert if_condition == {'attachments': {'media_keys': '123456'}, 'media': ['1', '2', '3', '4', '5', '6']}




def test_map_username_to_fweet(): 
  #AUTHOR ID AND ID SHOULD BE INTS
  test_fixture = {"author_id" : 12345}
  if_test_users = [{"id" : 12345, "name": "test_name", "profile_image_url" : "https://test_url", "verified": "yes", "location" : "test_location", "username" : "test_username"}]

  else_test_users = [{"id" : 7457235, "name": "test_name", "profile_image_url" : "https://test_url", "verified": "yes",   "location" : "test_location", "username" : "test_username"}] 

  if_test_result = twitter.map_usernames_to_fweet(test_fixture, if_test_users)
  assert if_test_result.get("name") == "test_name"
  assert if_test_result.get("profile_pic") == "https://test_url"
  assert if_test_result.get("verified") == "yes"
  assert if_test_result.get("location") == "test_location"
  assert if_test_result.get("username") == "test_username"
  
  #without the []
  assert if_test_result.get("mention_obj") == if_test_users[0]
  
  else_test_result = twitter.map_usernames_to_fweet(test_fixture, else_test_users) 
  assert else_test_result.get("mention_users").get("test_username") == else_test_users[0]


def test_upload_media(): 
  #can only test else condition because don't want to change mode to PROD 
  mode_fixture = "DEV"
  row_fixture = {}
  auth_fixture = ""

  result = twitter.upload_media(row_fixture, auth_fixture, mode_fixture)
  assert result == {"media_id" : None}



def test_send_tweet(): 
  row_fixture = {"id": "test_id", "media_id" : "test_media_id", "reply" : "test_reply"}
  mode_fixture = "DEV"
  auth_fixture = ""  

  test_fixture = twitter.send_tweet(row_fixture, auth_fixture, mode_fixture)
  assert test_fixture == {"id": "test_id", "media_id" : "test_media_id", "reply" : "test_reply"}   
  #returns same because prints statements - test those? 
  #otherwise cannot test PROD mode 




## Materializer
---

In [ ]:
#NOT TESTED
from google.cloud import storage
from pdf2image import convert_from_path

# TODO: move to kubelt SDK
class Materializer:
    mode = os.environ.get('FWEET_ENV', 'DEV')

    # Instantiates a client
    __bucket_name = "fweet"
    __data_path = 'data'
    #because of the different modes, need a lambda function to figure out the file path
    __fax_numbers_file_name = (lambda self :'%s/fax.csv' % self.__data_path)
    __last_exection_file_name = (lambda self :'%s/last.txt' % self.__data_path)
    __fweet_file_name = (lambda self :'%s/fweet.json' % self.__data_path)

    # Creates the new bucket
    __bucket = None

    def __init__(self):
        #TEST
        if self.mode == 'TEST':
            self.__data_path = 'fixtures'
        elif self.mode == 'PROD':
            # TODO: this can be replaced with an SDK pull and set
            # secrets = kubelt.Secrets(env='ENV')
            # secrets.get('fweet_bucket_creds.json', '%s/./config/gcloud/fweet_credentials.json' % pathlib.Path().home() )
            # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './secret/credentials.json'

            storage_client = storage.Client()
            self.__bucket = storage_client.bucket(self.__bucket_name)
        elif self.mode == 'DEV':
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './secret/credentials.json'

        
        # TODO: DEV mode will use /data path
    
    #for not in prod mode
    def __blob_download(self, filename):
        if self.mode != 'PROD': return
        pathlib.Path('%s/%s' % (os.getcwd(), self.__data_path)).mkdir(parents=True, exist_ok=True)
        blob = self.__bucket.blob(filename)
        blob.download_to_filename(filename)
        return blob

    def __blob_upload(self, filename):
        if self.mode != 'PROD': return
        blob = self.__bucket.blob(filename)
        blob.upload_from_filename(filename)
        return blob

    def get_fax_numbers(self):
        self.__blob_download(self.__fax_numbers_file_name())
        return pd.read_csv(self.__fax_numbers_file_name(), dtype={'number': 'string'})
        # return pd.read_csv(self.__fax_numbers_file_name()).to_dict(orient='list')
    
    #because it's a text file, just look at the last execution    
    def get_last_execution(self):
        self.__blob_download(self.__last_exection_file_name())
        with open(self.__last_exection_file_name(), 'r') as file:
            last_execution = file.read()
        return last_execution

    #typecast all the tweet info with the blob
    def get_fweet_df(self):
        self.__blob_download(self.__fweet_file_name())
        return pd.read_json(self.__fweet_file_name(), dtype={'id': 'int64', 'conversation_id': 'int64', 'fax_number': 'string'})
    
    def write_fax_numbers(self, df):
        df.astype({'number': 'string'})
        with open(self.__fax_numbers_file_name(), 'w') as file:
            file.write(df.to_csv(index=False))
            file.close()
        self.__blob_upload(self.__fax_numbers_file_name())

    def set_last_execution(self):
        with open(self.__last_exection_file_name(), 'w') as file:
            file.write(str(datetime.datetime.now()))
            file.close()
        self.__blob_upload(self.__last_exection_file_name())

    def write_fweet_df(self, df):
        with open(self.__fweet_file_name(), 'w') as file:
            file.write(df.to_json())
            file.close()
        self.__blob_upload(self.__fweet_file_name())

    def write_fax_pdf(self, filename, public=True):
        blob = self.__blob_upload(filename)
        signed_url = None
        if public and self.mode == 'PROD':
            signed_url = blob.generate_signed_url(expiration=datetime.timedelta(hours=1))
        return True, signed_url

    def write_jpg(self, jpg, public=True):
        blob = self.__blob_upload(jpg)
        signed_url = None
        if public and self.mode == 'PROD':
            signed_url = blob.generate_signed_url(expiration=datetime.timedelta(hours=1))
        return True, signed_url


## Fax Machine
---

In [ ]:
import telnyx
import re
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML, CSS
from weasyprint.fonts import FontConfiguration
from pathlib import Path

class FaxMachine:
    mode = os.environ.get('FWEET_ENV', 'DEV')

    rolodex = None 

    fweet_template = None
    font_config = None
    tcss = None
    wcss = None

    telnyx_app_id = '1617486772244055400'

    def __init__(self, materializer):
        #instantiated in the constructor
        self.rolodex = materializer.get_fax_numbers()
        telnyx.api_key = 'KEY0178E5E7B6C6DA17409A0E57C9ED18A3_EFFfwSbPAN6YpIDzsxlmHI'

        jinja_environment = Environment(
            loader=FileSystemLoader('%s/templates/' % workspace)
        )
        self.fweet_template = jinja_environment.get_template('fweet.html')
        self.font_config = FontConfiguration()
        self.tcss = CSS('%s/static/css/tailwind.css' % workspace)
        self.wcss = CSS('%s/static/css/all.css' % workspace, font_config=self.font_config)
    
    #just return rolodex
   #not tested
    def get_rolodex(self):
        return self.rolodex

    
    #not tested
    def has_balance(self):
        telnyx_balance = telnyx.Balance.retrieve()

        oc_balance_resp = requests.get('https://opencollective.com/fweet.json')
        oc_balance = oc_balance_resp.json()['balance']

        if float(telnyx_balance.available_credit) < 0.007 and float(oc_balance) < 0.01:
            logger.error('Out of ink! Please refill.')
            # TODO: send a tweet asking for ink refill with link to open collective
            return False
        return True

    
    
    @staticmethod
    #TESTED
    #called on run_set_instructions step, with each row of the df where the status value is null
    def set_instruction(row):
        #status == ?? None ??, just return as is??
        if row['status']:
            return row #wouldn't this always be true?
        if isinstance(row['entities.hashtags'], list):
            for hashtag in row['entities.hashtags']:
                if row['status']:
                    break
                if hashtag['tag'] == 'addfax':
                    row['status'] = 'addfax'
                elif hashtag['tag'] == 'fweet' and row['id'] != row['conversation_id']:
                    row['status'] = 'fweet'
        
        if not row['status']:
            if row['id'] != row['conversation_id']:
                row['status'] = 'fweet'
            else:
                # TODO: set a status to correctly let the user know they didn't use the hashtag right
                row['status'] = 'ignore'
        
        return row

           

    @staticmethod
    #not tested because just a setter method
    def set_status(row, status):
        row['status'] = status
        return row

    @staticmethod
    #TESTED
    #used in add fax_number, called with df where status==addfax
    def parse_fax_number(row, favorites_df):
        fax_user = None
        fax_number = None

        is_approved = False

        print('check approval')
        # print(row)
        #need to have one like
        print(row['public_metrics.like_count'] > 0)
        print(row['id'])
        print(favorites_df)


        if row['public_metrics.like_count'] > 0:
            # these are @fweetmachines favorites so the id_str is our id
            tweet_df = favorites_df[favorites_df['id_str'] == str(row['id'])]
            print(tweet_df)
            
            #only true when tweet_df is not empty
            is_approved = not tweet_df.empty

        if is_approved:
            for mention in row['entities.mentions']:
                if mention['username'] != 'fweetmachine':
                    fax_user = '@%s' % mention['username']
                    row['reply'] = 'Succesfully added @%s to fweet address book.' % mention['username']
           
            #phone number regex
            for match in re.findall(r"\(?\b[2-9][0-9]{2}\)?[-. ]?[2-9][0-9]{2}[-. ]?[0-9]{4}\b", row['text']):
                fax_number = phonenumbers.format_number(phonenumbers.parse(match, "CA"), phonenumbers.PhoneNumberFormat.E164)
            
            row['status'] = 'fax_added'
        else:
            # TODO: add logic to deny a "#addfax" instruction
            pass


        row['new_fax_username'] = fax_user
        row['new_fax_number'] =  fax_number
        return row
    
    @staticmethod
    #TESTED
    #applied in analyze mentions step, with rolodex, and net_new_df 
    #net_new_df is the df with the status 'fweet' 
    def map_fax(row, rolodex):
        records = list()
        for mention in row['entities.mentions']:
            username = mention['username']
            #PUT BACK deep=True
            record = row.copy()
            #if we are the one tweeting, skip, put append at the end with status skip
            if (username == 'fweetmachine'):
                record['status'] = 'skip'
                records.append(record)
                continue
            #add username
            record['fax_username'] = username
            try:
                # Assumes handles map to one fax number. If not, change here to iterate.
                #fax row first entry is the username
                fax_row = rolodex[rolodex['handle'] == '@%s' % username].iloc[0]
                print('====ROLODEX===')
                record['fax_number'] = str(fax_row.number)
                record['status'] = 'received'
                record['reason'] = '200 OK, ciao'
                record['reply'] = 'Fax being processed for %s.' % username
            except IndexError:
                # Still add with status fax_404
                record['fax_number'] = None
                record['status'] = 'denied'
                record['reason'] = '404 Not Found'
                record['reply'] = 'No fax number found for %s. \n\n Instructions to add a fax number is here: https://twitter.com/fweetmachine/status/1389675212573224962' % username

            records.append(record)
         #records is a list of records, which in itself is a list of usersname, fax_number, status, reason, reply   
        return records

    @staticmethod
    def run(row, fax_machine, materializer, mode, telnyx_mock=None):
        row['text'] = row['text'].strip('@fweetmachine')
        #writing pdf has the same problem as the jpg problem, so fix that 
        FAX_PDF = '%s/pdf/%s-%s.pdf' % (workspace, row['id'], row['fax_number'])
        Path(os.path.dirname(FAX_PDF)).mkdir(parents=True, exist_ok=True) 
        html = fax_machine.fweet_template.render(tweet=row)
        whtml = HTML(string=html, encoding='utf8')
        whtml.write_pdf(FAX_PDF, stylesheets=[fax_machine.wcss, fax_machine.tcss], font_config=fax_machine.font_config)

        print("===PHONE NUMBA===")
        print(row['fax_number'])
        print(type(row['fax_number']))

        #TODO: test these when the block below is refactored
        TO_FAX = phonenumbers.format_number(phonenumbers.parse(row['fax_number'], "CA"), phonenumbers.PhoneNumberFormat.E164)
        FROM_FAX = phonenumbers.format_number(phonenumbers.parse('+17093090259', "CA"), phonenumbers.PhoneNumberFormat.E164)
        APP_ID = fax_machine.telnyx_app_id
        _, FAX_URL = materializer.write_fax_pdf(FAX_PDF)

        # use this to materialize to bucket and then set a url before tweeting response
        #need to assert that this is the valid location on the file system, and if this is written correctly, 
        #it would test whether this is written correctly 
        

        row['pdf'] = FAX_PDF
     

        #TODO: this code needs to be refactored for testability 
        telnyx_fax = telnyx_mock
        print(telnyx_fax.status, "===STATUS=====")
        if (telnyx_fax):
            print("===FAX===")
            if mode != 'PROD':
                print('debug mode on. skipping fax...')
                #telnyx_fax = type('obj', (object,), {'status': 'queued'})
            else:
                telnyx_fax = telnyx.Fax.create(
                    from_=FROM_FAX,
                    to=TO_FAX,
                    media_url=FAX_URL,
                    connection_id=APP_ID,
                )
                print(telnyx_fax.__dict__)


        if telnyx_fax.status != 'queued':
            row['reason'] = 'This Fweet has failed to send'
            row['reply'] = 'Paper jam: %s' %telnyx_fax.status
            row['status'] = 'fax_failed'
            logger.warn('Error sending fax.')
        else:
            row['reason'] = 'Successfully sent.'
            row['reply'] = 'Fweet has been sent to @%s' %row['fax_username']
            row['status'] = 'sent'

        return row


In [ ]:
from unittest.mock import MagicMock, Mock, PropertyMock

def test_set_instructions():
    if_row_fixture = {"status": True}

    #would return immediately
    # if row['status']:
    #          return row
    test_else_fixture = fax.set_instruction(if_row_fixture)
    assert test_else_fixture == {"status": True }
    

    
    #                 row['status'] = 'ignore'
    #if isinstance(row['entities.hashtags'], list):
    #   if not row['status']:
    #             if row['id'] != row['conversation_id']:
    #                 row['status'] = 'fweet'
    if_row_fixture_entities =  {"status": False, "entities.hashtags" : "test_hashtag", "id" : 1234, "conversation_id" : 1234}
    test_if_row_fixture_entities = fax.set_instruction(if_row_fixture_entities)
    assert test_if_row_fixture_entities.get("status") == "ignore"

    if_row_fixture_entities_not_equal =  {"status": False, "entities.hashtags" : "test_hashtag", "id" : 1234, "conversation_id" : 4321}
    test_if_row_fixture_entities_not_equal = fax.set_instruction(if_row_fixture_entities_not_equal)
    assert test_if_row_fixture_entities_not_equal.get("status") == "fweet"

    
    #    for hashtag in row['entities.hashtags']:
    #                 if row['status']:
    #                     break

    if_row_fixture_entities_true = {"status": True, "entities.hashtags" : [{"tag" : "addfax"}]}
    test_if_row_fixture_entities_true = fax.set_instruction(if_row_fixture_entities_true)
    assert test_if_row_fixture_entities_true.get("status") == True
    

    ##if isinstance(row['entities.hashtags'], list):
    #if hashtag['tag'] == 'addfax':
    #                row['status'] = 'addfax'
    if_row_fixture_entities_false = {"status": False, "entities.hashtags" : [{"tag" : "addfax"}]}
    test_if_row_fixture_entities_false = fax.set_instruction(if_row_fixture_entities_false)
    assert test_if_row_fixture_entities_false.get("status") == "addfax"

    # elif hashtag['tag'] == 'fweet' and row['id'] != row['conversation_id']
    #                    row['status'] = 'fweet'
    if_row_fixture_entities_false_fweet = {"status": False, "entities.hashtags" : [{"tag" : "fweet"}], "id": 1234, "conversation_id" : 4321}
    test_if_row_fixture_entities_fweet = fax.set_instruction(if_row_fixture_entities_false_fweet)
    assert test_if_row_fixture_entities_fweet.get("status") == "fweet"


#because this function has branches, in order to test all the possible outcomes, we need to have different test fixtures


def test_fax_run():
    #(row, fax_machine, materializer, mode)
    row_fixture = {"text" : "@fweetmachine", "id": 123, "fax_number": "123", "fax_username": "test_username"}
    #because we need a fixture with functionality inside it, not just a dict, or int etc. we will need to use unittest.mock()
    html_fixture = "hello world"
    workspace_fixture = os.getcwd()
    fweet_template_mock = Mock()
    fweet_template_mock.render=MagicMock(return_value=html_fixture)

    fax_machine_fixture = Mock()
    fax_machine_fixture.font_config = FontConfiguration()
    fax_machine_fixture.tcss = CSS('%s/static/css/tailwind.css' % workspace_fixture)
    fax_machine_fixture.wcss = CSS('%s/static/css/all.css' % workspace_fixture, font_config=fax_machine_fixture.font_config)
    fax_machine_fixture.fweet_template=fweet_template_mock
    
     
    materializer_fixture = Mock()
    #just use magicmock 
    materializer_fixture.write_fax_pdf = MagicMock(return_value=(None, "dummy_fax_url"))
    mode_fixture = "DEV"


    # response_mock = Mock()
    # #response_mock.status_code = 200
    telnyx_mock = Mock()
    type(telnyx_mock).status = PropertyMock(return_value="queued")
   
    
    result_row_queued = fax.run(row_fixture, fax_machine_fixture, materializer_fixture, mode_fixture, telnyx_mock=telnyx_mock)
    #pdf is present, so we assume that since it exists, it read right
    assert ("pdf" in result_row_queued)
    assert result_row_queued["text"] == ""
    assert result_row_queued["reason"] == "Successfully sent."
    #shouldn't make a diff if you put an @ or not
    assert result_row_queued["reply"] == 'Fweet has been sent to @%s' %result_row_queued['fax_username']
    assert result_row_queued['status'] == 'sent'
    

    telnyx_mock = Mock()
    type(telnyx_mock).status = PropertyMock(return_value="not_queued")
    

    result_row_not_queued = fax.run(row_fixture,fax_machine_fixture, materializer_fixture, mode_fixture, telnyx_mock)
    assert ("pdf" in result_row_queued)
    assert result_row_not_queued["text"] == ""
    assert result_row_not_queued["reason"] == 'This Fweet has failed to send'
    #this should read paper jam: not queued because we're calling it on the mock 
    assert result_row_not_queued["reply"] == 'Paper jam: %s' %telnyx_mock.status
    assert result_row_not_queued['status'] == 'fax_failed'




def test_parse_fax_number(): 
   
    fav_fixture = pd.DataFrame({"id_str" : "1234", True: "True"}, index=[0])
    # if row['public_metrics.like_count'] > 0:
#             # these are @fweetmachines favorites so the id_str is our id
#             tweet_df = favorites_df[favorites_df['id_str'] == str(row['id'])]
#             print(tweet_df)
    #having it as an int will test the type casting as well 
    if_row_fixture = {"public_metrics.like_count" : 1, "id" : 1234, "entities.mentions" : [{"username": "not_fweet_machine"}],  "text" : "test_text"}
    else_row_fixture = {"public_metrics.like_count" : -3, "id" : 1234, "entities.mentions" : [{"username": "fweetmachine"}],  "text" : "test_text" }

#only true if fweet_df is not empty, and not empty when the if statement is True
# if is_approved:
#             for mention in row['entities.mentions']:
#                 if mention['username'] != 'fweetmachine':
#                     fax_user = '@%s' % mention['username']
#                     row['reply'] = 'Succesfully added @%s to fweet address book.' % mention['username']
#             #phone number regex
#             for match in re.findall(r"\(?\b[2-9][0-9]{2}\)?[-. ]?[2-9][0-9]{2}[-. ]?[0-9]{4}\b", row['text']):
#                 fax_number = phonenumbers.format_number(phonenumbers.parse(match, "CA"), phonenumbers.PhoneNumberFormat.E164)
            
#             row['status'] = 'fax_added'

    test_both_if = fax.parse_fax_number(if_row_fixture, fav_fixture)
    assert test_both_if.get("reply") == 'Succesfully added @%s to fweet address book.' % if_row_fixture.get("entities.mentions")[0].get("username")
    assert test_both_if.get("status") == 'fax_added'
 
    
#         else:
#             # TODO: add logic to deny a "#addfax" instruction
#             pass
    test_else = fax.parse_fax_number(else_row_fixture, fav_fixture) 
    assert test_else.get("new_fax_username") == None
    assert test_else.get("new_fax_number") == None
   
   
def test_map_fax():
    data_fixture = {"fax_username" : "not_fweet_machine", "number" : 54321}
    rolodex_fixture = pd.Series({"handle" : "not_fweet_machine", False: pd.DataFrame(data=data_fixture, index=[0])
    })
    rolodex_fixture_true = pd.Series({"handle" : "not_fweet_machine", True: pd.DataFrame(data=data_fixture, index=[0])
    })
    if_row_fixture = {"entities.mentions" : [{"username" : "fweetmachine"}]}
    else_row_fixture = {"entities.mentions" : [{"username" : "not_fweet_machine"}]}
    
#         for mention in row['entities.mentions']:
#             username = mention['username']
#             record = row.copy(deep=True)
#             #if we are the one tweeting, skip, put append at the end with status skip
#             if (username == 'fweetmachine'):
#                 record['status'] = 'skip'
#                 records.append(record)
#                 continue
    
    test_if_fixture = fax.map_fax(if_row_fixture, rolodex_fixture)
    assert test_if_fixture[0].get("status") == 'skip'

    test_else_fixture = fax.map_fax(else_row_fixture, rolodex_fixture)
    print(test_else_fixture)
    assert test_else_fixture[0].get("fax_username") == 'not_fweet_machine'
    assert test_else_fixture[0].get("status") == 'received'
    assert test_else_fixture[0].get("reason") == '200 OK, ciao'
    assert test_else_fixture[0].get('reply') == 'Fax being processed for %s.' % test_else_fixture[0].get("entities.mentions")[0].get('username')

    #test except block
    
    # test_else_fixture_true = fax.map_fax(else_row_fixture, rolodex_fixture_true)
    # test_else_fixture_true.assertRaises(IndexError)
    # assert test_else_fixture_true[0].get("fax_number") == None
    # assert test_else_fixture_true[0].get("status") == 'denied'
    # assert test_else_fixture_true[0].get("reason") == '404 Not found'
    # assert test_else_fixture_true[0].get("reply") == 'No fax number found for %s. \n\n Instructions to add a fax number is here: https://twitter.com/fweetmachine/status/1389675212573224962' % test_else_fixture_true[0].get("entities.mentions")[0].get('username')
     



# Fweet Machine App

---

## Process Sent

In [ ]:
def process_sent(sent_df, completed_df):
    # poll telynx
    return sent_df, completed_df

## Process Faxes

In [ ]:
from pathlib import Path

#TESTED
#called in faxing_df, where if the fax has balance, apply convert from path to each row, 
#turn the pdf header to jpg, limit 500
def generate_jpg_from_pdf(row):
    pages = convert_from_path(row['pdf'], 500)
    #iterating page correctly?
    for page in pages:
        #building path correctly?
        jpg = '%s/jpg/%s.jpg' % (workspace, os.path.splitext(os.path.basename(row['pdf']))[0])
        #writing the jpg correctly, we know that this does not work when no jpg folder in dir, 
        #so a way to make it better is to use a pythonlib called pathlib
        #make sure path exists
        Path(os.path.dirname(jpg)).mkdir(parents=True, exist_ok=True) 
        page.save(jpg, 'JPEG')

        _, signed_url = materializer.write_jpg(jpg)

        #return a df with jpg_url, and jpg 
        #mutate row correctly?
        row['jpg_url'] = signed_url
        row['jpg'] = jpg
        return row

In [ ]:
def test_generate_jpg_from_pdf(): 
    
    import urllib

    fixture_row = pd.Series({"pdf" : "./fixtures/1387474633679585280.pdf"})
    result_row = generate_jpg_from_pdf(fixture_row)
    assert ("jpg_url" in result_row)
    assert ("jpg" in result_row)
    assert os.path.exists(result_row['jpg'])
    assert urllib.parse.urlparse(result_row['jpg_url'])


In [ ]:
#used in run process fax step,for status == pending, checks to see if it has money, and then send tweet

#Not tested because the methods called within are already tested 
def process_fax(faxing_df):

    if (not fax.has_balance()):
        # TODO: send a tweet that we are are out of ink and to donate (once every 4)
        return

    faxing_df = faxing_df.apply(fax.run, args=(fax,materializer,fax.mode), axis=1)

    faxing_df = faxing_df.apply(generate_jpg_from_pdf, axis=1)
    #returns a df with jpg, and jpg url (jpg of the fax that needs to be sent)
    faxing_df = faxing_df.apply(twitter.upload_media, args=(twitter.auth, twitter.mode,), axis=1)
    faxing_df.apply(twitter.send_tweet, args=(twitter.auth, twitter.mode,), axis=1)

    return faxing_df


## Process Pending

In [ ]:
#used in process pending step, if status is pendig, then update status and popluate reason, reply


#TESTED
def process_pending(pending_df):
    # Move recieved to pending status (straight assignment because there is currently no review process)
    # TODO: some approval criteria (e.g. public metrics)
    """
    Some validation ideas:
    - NFT (new fax token with 50% resell comish)
    - Only Fweet Tweets from users that follow @fweet machine
    - Possibly so like and retweet criteria
    """

    # TODO: use state transition function
    pending_df.loc[pending_df['status'] == 'received', 'reason'] = 'Fweet has been approved' 
    pending_df.loc[pending_df['status'] == 'received', 'reply'] = 'Your fax is now queued for sending.' 
    pending_df.loc[pending_df['status'] == 'received', 'status'] = 'pending' 

    #is this just good practise? because won't ever get here??
    pending_df.loc[pending_df['status'] == 'denied', 'status'] = 'dead letter' 

    return pending_df

In [ ]:
def test_pending_df():
    pending_df_fixture_recieved = pd.DataFrame({"status" : "received"}, index=[0])
    pending_df_fixture_denied = pd.DataFrame({"status" : "denied"}, index=[0])

    result = process_pending(pending_df_fixture_recieved)
    #assert result ==     status                   reason                                reply
    #0  pending  Fweet has been approved  Your fax is now queued for sending.

    denied = process_pending(pending_df_fixture_denied)
#     assert denined ==       status reason reply
#     0  dead letter    NaN   NaN


## Analyze Mentions

In [ ]:
#TESTED

def analyze_mentions(fweet_df, media, users): 
    net_new_df = fweet_df[fweet_df['status'] == 'fweet'].astype({'fax_number': 'string'})
    not_net_new_df = fweet_df[fweet_df['status'] != 'fweet'].astype({'fax_number': 'string'})
    
    # Look up fax numbers for everything without a status because we haven't operated on it yet.
    # if the df has status fweet, then apply map_fax, which return a list of records to the new df
    #will always return, either records or error
    net_new_rows = net_new_df.apply(fax.map_fax, args=(fax.get_rolodex(),), axis=1)
    
    #only if no new fweet status
    if net_new_rows.empty:
        return fweet_df

    #squash the list of lists of lists, into one    
    username_rows = [item for sublist in net_new_rows for item in sublist]
    #username_df rows, with the fax number as a string
    username_df = pd.DataFrame(data=username_rows).astype({'fax_number': 'string'})

    # Get user and media data for new users.
    # now, look at the username_df, and if the name feild is null, which means these are new tweets
    new_users_df = username_df[username_df['name'].isnull()]
    # populated_new_users_df = new_users_df.apply(twitter.map_media_to_tweet, args=(media,), axis=1)

    #adds the user information into the df
    populated_new_users_df = new_users_df.apply(twitter.map_usernames_to_fweet, args=(users,), axis=1)

    # Add the new data back to the received dataframe.
    #return df
    fweet_df = pd.concat([not_net_new_df, populated_new_users_df]).reset_index(drop=True)

    return fweet_df

In [ ]:
#test the fail state, because map_username_to_fweet has been tested
def test_analyze_mentions():
    media_fixture = {}
    user_fixture = {}
    fweet_df_fixture_false = pd.DataFrame({"status" : "not_fweet", "fax_number": 1234}, index=[0])

    #should just get the df back
    not_new_result = analyze_mentions(fweet_df_fixture_false, media_fixture, user_fixture)
    # assert now_new_result ==       status  fax_number
    #                             0  not_fweet        1234    


## Steps

In [ ]:
def run_bootstrap_step():
    print('mode')
    print(os.environ.get('FWEET_ENV', 'DEV'))

    # bootstrap twitter
    global twitter
    twitter = Twitter()
    twitter.load_bearer_token()

    global materializer
    materializer = Materializer()

    global fax
    fax = FaxMachine(materializer)


In [ ]:
#don't need to test because tested set_status, and get_tweet_content
def run_received_step(fweet_df, media, users):
    fweet_df = analyze_mentions(fweet_df, media, users)
        
    # Respond to net new Tweets in the updated recieved dataframe
    failed_df = fweet_df[(fweet_df['status'] == 'denied') | (fweet_df['status'] == 'failed')]
    not_failed_df = fweet_df[(fweet_df['status'] != 'denied') | (fweet_df['status'] != 'failed')]

    print("===GETTING CONTENT====")
    print(not_failed_df[['id', 'conversation_id', 'status', 'text']])

    failed_df['content'] = None
    failed_df = failed_df.apply(fax.set_status, args=('denied_complete',), axis=1)
    # not failed df, if contains not rows with status pending or recieved, just returns the same df
    # so, if the tweet has not been denied or failed, and the status is pending or recieved, populate the df the all the information of the tweet and put in this df
    not_failed_df = not_failed_df.apply(twitter.get_tweet_content, args=(twitter.bearer_token,), axis = 1)

    #concat everything, and return
    fweet_df = pd.concat([not_failed_df, failed_df]).reset_index(drop=True)
    
    # write back
    materializer.write_fweet_df(fweet_df)

    return fweet_df

In [ ]:
def run_approval_step(fweet_df):
    pending_df = fweet_df[fweet_df['status'] == 'received']
    not_pending_df = fweet_df[fweet_df['status'] != 'received']

    # pending_df = pending_df.reset_index(drop=True).truncate(before=-1, after=-1)
    # pending_df['id'] = None
    # pending_df['fax_username'] = None

    #update reason, reply for status == recieved
    #set reply, and reason and status as pending
    pending_df = process_pending(pending_df)

    #concat those
    fweet_df = pd.concat([not_pending_df, pending_df]).reset_index(drop=True)

    #write back
    materializer.write_fweet_df(fweet_df)

    return fweet_df

In [ ]:
def run_fax_step(fweet_df):
    faxing_df = fweet_df[fweet_df['status'] == 'pending']
    not_faxing_df = fweet_df[fweet_df['status'] != 'pending']
    
    #returns a df with row jpg, and jpg url 
    faxing_df = process_fax(faxing_df)

    #concat with the new status of the df and return
    fweet_df = pd.concat([not_faxing_df, faxing_df]).reset_index(drop=True)

    # write back
    materializer.write_fweet_df(fweet_df)

    return fweet_df

In [ ]:
#not tested because materializer funcs are invoked
def run_complete_step():
    sent_df = materializer.get_sent_df()
    completed = materializer.get_completed_df()

    sent_df, completed_df = process_sent(sent_df, completed_df)

In [ ]:
#TESTED
def run_add_fax_number_step(fweet_df):
    favorites_df = twitter.fetch_favorites()
    add_fax_df = fweet_df[fweet_df['status'] == 'addfax']
    not_add_fax_df = fweet_df[fweet_df['status'] != 'addfax']

    #adds fax number and username to where status addfax
    add_fax_df = add_fax_df.apply(fax.parse_fax_number, args=(favorites_df,), axis=1)
    #so all the new fax_number and usernames are then turned to a new df
    new_fax_df = add_fax_df[(add_fax_df['new_fax_username'].notnull()) & (add_fax_df['new_fax_number'].notnull()) ]


    cols = list(fax.rolodex.columns)
    new_df = new_fax_df[['new_fax_username', 'new_fax_number']]
    # rename columns to rolodex colums
    new_df.columns = cols

    new_df = new_df.drop_duplicates()

    #not already in the rolodex
    net_new = new_df[(~new_df.handle.isin(fax.rolodex.handle))&(~new_df.number.isin(fax.rolodex.number))] 

    fax.rolodex = pd.concat([fax.rolodex,net_new])

    materializer.write_fax_numbers(fax.rolodex)

    #put all this is the fweetdf, and return
    fweet_df = pd.concat([not_add_fax_df, add_fax_df]).reset_index(drop=True)

    # send a tweet that fax has been added
    add_fax_df.apply(twitter.send_tweet, args=(twitter.auth, twitter.mode,), axis=1)

    return fweet_df



In [ ]:
def test_run_add_fax_number():
    add_fax_df_fixture = pd.DataFrame({"status" : "addfax", "new_fax_username" : "test_username", "new_fax_number" : "test_number", "public_metrics.like_count" : 10, "id" : 12345, "reply" : "test_reply"}, index=[0])
    #false statement
    not_add_fax_df_fixture = pd.DataFrame({"status" : "not_add_fax", "new_fax_username" : "test_username", "new_fax_number" : "test_number"}, index=[0])

    #where status is not_add_fax, so should return the same thing
    false_fixture = run_add_fax_number_step(not_add_fax_df_fixture)


#     assert false_fixture == 
#         status new_fax_username new_fax_number
# 0  not_add_fax    test_username    test_number



#TEST TRUE CONDITION - commented out because of whitespace


#     true_fixture = run_add_fax_number_step(add_fax_df_fixture)
#     assert true_fixture ==                        created_at                   id               id_str  \
# 0   Sat May 08 16:40:47 +0000 2021  1391070586823221250  1391070586823221250   
# 1   Sat May 08 16:40:24 +0000 2021  1391070489574060034  1391070489574060034   
# 2   Sat May 08 16:39:57 +0000 2021  1391070378575810562  1391070378575810562   
# 3   Sat May 08 16:39:24 +0000 2021  1391070241145430018  1391070241145430018   
# 4   Sat May 08 16:38:59 +0000 2021  1391070135637716992  1391070135637716992   
# 5   Sat May 08 16:38:35 +0000 2021  1391070032105558022  1391070032105558022   
# 6   Sat May 08 16:38:06 +0000 2021  1391069912257466373  1391069912257466373   
# 7   Sat May 08 16:36:45 +0000 2021  1391069573026304003  1391069573026304003   
# 8   Sat May 08 16:36:22 +0000 2021  1391069476033019908  1391069476033019908   
# 9   Sat May 08 16:35:56 +0000 2021  1391069364703612936  1391069364703612936   
# 10  Sat May 08 16:35:29 +0000 2021  1391069253223206912  1391069253223206912   
# 11  Sat May 08 16:35:06 +0000 2021  1391069156791947264  1391069156791947264   
# 12  Sat May 08 16:34:44 +0000 2021  1391069063275745284  1391069063275745284   
# 13  Sat May 08 16:34:21 +0000 2021  1391068967167565827  1391068967167565827   
# 14  Sat May 08 16:33:36 +0000 2021  1391068779795341313  1391068779795341313   
# 15  Sat May 08 16:33:11 +0000 2021  1391068673360678912  1391068673360678912   
# 16  Sat May 08 16:32:47 +0000 2021  1391068573351677954  1391068573351677954   
# 17  Sat May 08 16:32:22 +0000 2021  1391068467923718155  1391068467923718155   
# 18  Sat May 08 16:31:50 +0000 2021  1391068336595902472  1391068336595902472   
# 19  Sat May 08 16:31:20 +0000 2021  1391068210334732289  1391068210334732289   

#                                                  text  truncated  \
# 0   @fweetmachine @SameerZuberi\n613-996-8478 http...      False   
# 1   @fweetmachine @bobzimmermp\n613-947-4527 https...      False   
# 2   @fweetmachine @ZannLenore\n613-992-7220 https:...      False   
# 3   @fweetmachine @SalmaZahid15\n613-943-1045 http...      False   
# 4   @fweetmachine @DavidYurdiga\n613-992-4603 http...      False   
# 5   @fweetmachine @KateYoungMP\n613-996-6772 https...      False   
# 6   @fweetmachine @JeanYip3\n613-995-1612 https://...      False   
# 7   @fweetmachine @Puglaas\n613-992-1460 https://t...      False   
# 8   @fweetmachine @JohnWilliamson_\n613-995-5226 h...      False   
# 9   @fweetmachine @JonathanWNV\n613-992-7319 https...      False   
# 10  @fweetmachine @PatrickBWeiler\n613-947-4620 ht...      False   
# 11  @fweetmachine @Webber4Confed\n613-992-2537 htt...      False   
# 12  @fweetmachine @KevinWaugh_CPC\n613-995-0126 ht...      False   
# 13  @fweetmachine @chriswarkentin\n613-947-4782 ht...      False   
# 14  @fweetmachine @cathayw\n 613-992-8676 https://...      False   
# 15  @fweetmachine @BradleyVis\n613-992-1298 https:...      False   
# 16  @fweetmachine @viraniarif\n613-995-1629 https:...      False   
# 17  @fweetmachine @JulieVignolaBL\n613-992-4544 ht...      False   
# 18  @fweetmachine @ArnoldViersen\n613-995-1415 htt...      False   
# 19  @fweetmachine @GaryAVidal\n613-995-7697 https:...      False   

#                                                source in_reply_to_status_id  \
# 0   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 1   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 2   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 3   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 4   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 5   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 6   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 7   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 8   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 9   <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 10  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 11  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 12  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 13  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 14  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 15  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 16  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 17  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 18  <a href="https://mobile.twitter.com" rel="nofo...                  None   
# 19  <a href="https://mobile.twitter.com" rel="nofo...                  None   

#    in_reply_to_status_id_str  in_reply_to_user_id in_reply_to_user_id_str  \
# 0                       None  1383517092553510913     1383517092553510913   
# 1                       None  1383517092553510913     1383517092553510913   
# 2                       None  1383517092553510913     1383517092553510913   
# 3                       None  1383517092553510913     1383517092553510913   
# 4                       None  1383517092553510913     1383517092553510913   
# 5                       None  1383517092553510913     1383517092553510913   
# 6                       None  1383517092553510913     1383517092553510913   
# 7                       None  1383517092553510913     1383517092553510913   
# 8                       None  1383517092553510913     1383517092553510913   
# 9                       None  1383517092553510913     1383517092553510913   
# 10                      None  1383517092553510913     1383517092553510913   
# 11                      None  1383517092553510913     1383517092553510913   
# 12                      None  1383517092553510913     1383517092553510913   
# 13                      None  1383517092553510913     1383517092553510913   
# 14                      None  1383517092553510913     1383517092553510913   
# 15                      None  1383517092553510913     1383517092553510913   
# 16                      None  1383517092553510913     1383517092553510913   
# 17                      None  1383517092553510913     1383517092553510913   
# 18                      None  1383517092553510913     1383517092553510913   
# 19                      None  1383517092553510913     1383517092553510913   

#     ... user.profile_text_color user.profile_use_background_image  \
# 0   ...                  333333                              True   
# 1   ...                  333333                              True   
# 2   ...                  333333                              True   
# 3   ...                  333333                              True   
# 4   ...                  333333                              True   
# 5   ...                  333333                              True   
# 6   ...                  333333                              True   
# 7   ...                  333333                              True   
# 8   ...                  333333                              True   
# 9   ...                  333333                              True   
# 10  ...                  333333                              True   
# 11  ...                  333333                              True   
# 12  ...                  333333                              True   
# 13  ...                  333333                              True   
# 14  ...                  333333                              True   
# 15  ...                  333333                              True   
# 16  ...                  333333                              True   
# 17  ...                  333333                              True   
# 18  ...                  333333                              True   
# 19  ...                  333333                              True   

#    user.has_extended_profile user.default_profile user.default_profile_image  \
# 0                       True                 True                      False   
# 1                       True                 True                      False   
# 2                       True                 True                      False   
# 3                       True                 True                      False   
# 4                       True                 True                      False   
# 5                       True                 True                      False   
# 6                       True                 True                      False   
# 7                       True                 True                      False   
# 8                       True                 True                      False   
# 9                       True                 True                      False   
# 10                      True                 True                      False   
# 11                      True                 True                      False   
# 12                      True                 True                      False   
# 13                      True                 True                      False   
# 14                      True                 True                      False   
# 15                      True                 True                      False   
# 16                      True                 True                      False   
# 17                      True                 True                      False   
# 18                      True                 True                      False   
# 19                      True                 True                      False   

#     user.following  user.follow_request_sent  user.notifications  \
# 0            False                     False               False   
# 1            False                     False               False   
# 2            False                     False               False   
# 3            False                     False               False   
# 4            False                     False               False   
# 5            False                     False               False   
# 6            False                     False               False   
# 7            False                     False               False   
# 8            False                     False               False   
# 9            False                     False               False   
# 10           False                     False               False   
# 11           False                     False               False   
# 12           False                     False               False   
# 13           False                     False               False   
# 14           False                     False               False   
# 15           False                     False               False   
# 16           False                     False               False   
# 17           False                     False               False   
# 18           False                     False               False   
# 19           False                     False               False   

#     user.translator_type  user.withheld_in_countries  
# 0                   none                          []  
# 1                   none                          []  
# 2                   none                          []  
# 3                   none                          []  
# 4                   none                          []  
# 5                   none                          []  
# 6                   none                          []  
# 7                   none                          []  
# 8                   none                          []  
# 9                   none                          []  
# 10                  none                          []  
# 11                  none                          []  
# 12                  none                          []  
# 13                  none                          []  
# 14                  none                          []  
# 15                  none                          []  
# 16                  none                          []  
# 17                  none                          []  
# 18                  none                          []  
# 19                  none                          [] 




In [ ]:
#TESTED
def run_set_instruction_step(fweet_df, mentions_df):
    # Set up the mapping for our aggregation functions.
    # By default all columns are 'first',
    # public_metrics fields are 'max',
    # This helps us determine net new mentions since last run.
    
    columns = list(fweet_df.columns) + list(mentions_df.columns)

    mapping = dict()

    #set the mapping[column of fweet and mention df] value to first
    for column in columns:
        mapping[column] = 'first'


    mapping['public_metrics.retweet_count'] = max
    mapping['public_metrics.reply_count'] = max
    mapping['public_metrics.like_count'] = max
    mapping['public_metrics.quote_count'] = max


    mentions_df = mentions_df.astype({'id': 'int64'})

    # Combine the old mentions with the received new ones so we're operating on everything.
    # TODO: assign a fweet_id (uuidv4)
    updated_concat_df = pd.concat([
        fweet_df, 
        #if the id is not the same, concat the two dfs
        mentions_df[~mentions_df.id.isin(fweet_df.id)]
    ])

    # Aggregate the combined old and new mentions to find what's net new and update public_metrics.
    merged_fweet_df = updated_concat_df.groupby(by=['id']).agg(mapping, axis=1).reset_index(drop=True)

    # Look up fax numbers for everything without a status because we haven't operated on it yet.
    net_new_rows = merged_fweet_df[merged_fweet_df['status'].isnull()]
    not_new_rows = merged_fweet_df[~merged_fweet_df['status'].isnull()]

    
    if net_new_rows.empty:
        return fweet_df


    #?????????????? question
    applied_new_rows = net_new_rows.apply(fax.set_instruction, axis=1) # if row status is null, it just returns the row??

    fweet_df = pd.concat([not_new_rows, applied_new_rows]).reset_index(drop=True)
    
    return fweet_df

In [ ]:
def test_set_instructions():
    fweet_df_fixture_false = pd.DataFrame({"id" : "fweet_id", "fweet_col_1" : "fweet_test", "fweet_col_2": "fweet_test2", "status" : numpy.NaN}, index=[0])
    fweet_df_fixture_true = pd.DataFrame({"id" : "fweet_id", "fweet_col_1" : "fweet_test", "fweet_col_2": "fweet_test2", "status" : "value"}, index=[0])
    mentions_df_fixture = pd.DataFrame({"mentions_col_1" : "mentions_test", "mentions_col_2": "mentions_test2", "mentions_col_3" : "mentions_test", "id" : 1236127763, "public_metrics.like_count" : 1, "public_metrics.quote_count" : 10, 'public_metrics.reply_count' : 20, 'public_metrics.retweet_count' : 50, 'entities.hashtags' : ["test_vals"], "conversation_id" : 387167126}, index=[0])

    #should just return concat results
    false_result = run_set_instruction_step(fweet_df_fixture_false, mentions_df_fixture)
    # assert false_result ==             id fweet_col_1  fweet_col_2  status mentions_col_1  mentions_col_2  \
    #                                     0  1236127763        None         None     NaN  mentions_test  mentions_test2   
    #                                     1    fweet_id  fweet_test  fweet_test2     NaN           None            None   

    #                                     mentions_col_3  public_metrics.like_count  public_metrics.quote_count  \
    #                                     0  mentions_test                        1.0                        10.0   
    #                                     1           None                        NaN                         NaN   

    #                                     public_metrics.reply_count  public_metrics.retweet_count  
    #                                     0                        20.0                          50.0  
    #                                     1                         NaN                           NaN 

    #                                     public_metrics.reply_count  public_metrics.retweet_count  
    #                                     0                        20.0                          50.0  
    #                                     1                         NaN                           NaN 




    true_result = run_set_instruction_step(fweet_df_fixture_true, mentions_df_fixture)


#    assert true_result == 
#                             id fweet_col_1  fweet_col_2 status mentions_col_1  mentions_col_2  \
#                     0    fweet_id  fweet_test  fweet_test2  value           None            None   
#                     1  1236127763        None         None  fweet  mentions_test  mentions_test2   

#                     mentions_col_3  public_metrics.like_count  public_metrics.quote_count  \
#                     0           None                        NaN                         NaN   
#                     1  mentions_test                        1.0                        10.0   

#                     public_metrics.reply_count  public_metrics.retweet_count entities.hashtags  \
#                     0                         NaN                           NaN              None   
#                     1                        20.0                          50.0         test_vals   

#                     conversation_id  
#                     0              NaN  
# 1      387167126.0 




# RUN
---

In [ ]:
run_bootstrap_step()

##call all test functions only if in DEV mode
#if (os.environ.get('FWEET_ENV', 'DEV') == "DEV"):
    # test_fetch_mentions()
    #test_fetch_favorites() 
    # test_map_media_to_tweet()
    #test_get_tweet_content()
    #test_map_media_to_tweet()
    # test_map_username_to_fweet()
    #test_upload_media()
    #test_send_tweet()
    # test_set_instructions()
    # test_fax_run()
    #test_parse_fax_number()
    # test_map_fax()
    #test_pending_df()
    #test_generate_jpg_from_pdf()
    #test_analyze_mentions()
    #test_run_add_fax_number()
    #test_set_instructions()


fweet_df = materializer.get_fweet_df()
mentions_df, media, users = twitter.fetch_mentions()

fweet_df = run_set_instruction_step(fweet_df, mentions_df)

# # TODO: the next steps can run in parallel when supported

# #add new fax numbers to the rolodex
fweet_df = run_add_fax_number_step(fweet_df)

# #for fweets whose status is pending or recieved, populate all the information of the users, and raise error id denied
fweet_df = run_received_step(fweet_df, media, users)

# #set status as recieved, reason and reply for df that has been recieved and set status as pending, and for those that have been denied, as dead letter
fweet_df = run_approval_step(fweet_df)

# #send out the tweets for the pending status, if the fax machine has money, with pdf of fax turned to jpeg
fweet_df = run_fax_step(fweet_df)

# # TODO this step should remove compelte (success or not) to a respective blob
# #just adds the completed status to the fax 
fweet_df = run_completed_step()